This script is for updating Python formulas and "hide_until_python" properties of the document fields after some field codes have been changes.

Input: CSV log of the field code changes either provided by "Documents: Fix field codes" admin task or created manually.
Results: Python "formula" and "hide_until_python" scripts of all fields of each changed document type are updated to contain the new field codes instead of the old ones.

Warning: This script simply replaces substring (old field codes) occurencies in the formulas. It may damage a formula if it container the field code in a string constant or in similar cases.

In [1]:
import os
import io
import csv
import sys
import jiphy
from collections import defaultdict
sys.path.append('../../')
os.environ["DJANGO_SETTINGS_MODULE"] = "settings"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")
import django
from django.db import connection, transaction
django.setup()
from apps.document.models import DocumentField, DocumentType

In [2]:
# Columns should be: Document Type Code, Old Field Code, New Field Code
field_code_changes_csv = '''Document Type Code,Old Field Code,New Field Code
lease.LeaseDocument,alterationsAllowed,alterations_allowed
contract.SellersWarrantiesAndServ,swarrantiesOffCertificate,swarranties_offcertificate
lease.LeaseDocument,leaseType,lease_type
'''

In [3]:
f = io.StringIO(field_code_changes_csv)
reader = csv.reader(f, delimiter=',')

doc_types_to_changes = defaultdict(dict)

header = None
for row in reader:
    if header is None:
        header = row
    else:
        doc_type_code, old_field_code, new_field_code = row
        doc_types_to_changes[doc_type_code][old_field_code] = new_field_code

print(doc_types_to_changes)

defaultdict(<class 'dict'>, {'lease.LeaseDocument': {'alterationsAllowed': 'alterations_allowed', 'leaseType': 'lease_type'}, 'contract.SellersWarrantiesAndServ': {'swarrantiesOffCertificate': 'swarranties_offcertificate'}})


In [4]:
with transaction.atomic():
    for doc_type_code, changes in doc_types_to_changes.items():
        doc_type = DocumentType.objects.get(code=doc_type_code)
        
        for field in DocumentField.objects.filter(document_type=doc_type):
            formula = field.formula
            if formula:
                for old_code, new_code in changes.items():
                    formula = formula.replace(old_code, new_code)
                
                if field.formula != formula:
                    field.formula = formula
                    field.save(update_fields={'formula'})
                    print('Updated formula for {0}.{1}'.format(doc_type_code, field.code))
            
            hide_until = field.hide_until_python
            if hide_until:
                for old_code, new_code in changes.items():
                    hide_until = hide_until.replace(old_code, new_code)
                
                if field.hide_until_python != hide_until:
                    field.hide_until_python = hide_until
                    field.hide_until_js = jiphy.to.javascript(field.hide_until_python) if field.hide_until_python else ''
                    field.save(update_fields={'hide_until_python', 'hide_until_js'})
                    print('Updated hide_until_python for {0}.{1}'.format(doc_type_code, field.code))

(0.002) SELECT "document_documenttype"."created_date", "document_documenttype"."modified_date", "document_documenttype"."created_by_id", "document_documenttype"."modified_by_id", "document_documenttype"."uid", "document_documenttype"."code", "document_documenttype"."title", "document_documenttype"."field_code_aliases", "document_documenttype"."editor_type", "document_documenttype"."metadata" FROM "document_documenttype" WHERE "document_documenttype"."code" = 'lease.LeaseDocument'; args=('lease.LeaseDocument',)
(0.002) SELECT "document_documentfield"."created_by_id", "document_documentfield"."modified_by_id", "document_documentfield"."uid", "document_documentfield"."document_type_id", "document_documentfield"."code", "document_documentfield"."long_code", "document_documentfield"."title", "document_documentfield"."description", "document_documentfield"."type", "document_documentfield"."text_unit_type", "document_documentfield"."value_detection_strategy", "document_documentfield"."python_

Updated hide_until_python for lease.LeaseDocument.alterations_allowed
